# ERA5 mean 2m temperature

## Import packages

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
year_start = 1940
year_stop = 2022

# Climatology periods
climatologies = (slice("1941", "1980"), slice("1981", "2020"))
assert len(climatologies) == 2

## Define request

In [ ]:
collection_id = "reanalysis-era5-single-levels-monthly-means"
request = {
    "product_type": "monthly_averaged_reanalysis",
    "variable": "2m_temperature",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
    "time": "00:00",
    "format": "grib",
}

## Functions to cache

In [ ]:
def time_sliced_reduction(ds, time_slice, func, **kwargs):
    ds = ds.sel(forecast_reference_time=time_slice)
    return func(ds, **kwargs)

## Download and transform

In [ ]:
clima_dataarrays = []
month_dataarrays = []
season_dataarrays = []
for clima_slice in [slice(str(year_start), str(year_stop))] + list(climatologies):
    for func, da_list in zip(
        (
            diagnostics.time_weighted_mean,
            diagnostics.monthly_weighted_mean,
            diagnostics.seasonal_weighted_mean,
        ),
        (clima_dataarrays, month_dataarrays, season_dataarrays),
    ):
        print(f"{clima_slice=} {func.__name__=}")
        ds = download.download_and_transform(
            collection_id,
            request,
            chunks={"year": 1},
            transform_chunks=False,
            transform_func=time_sliced_reduction,
            transform_func_kwargs={
                "time_slice": clima_slice,
                "func": func,
                "weights": False,
            },
        )
        da = ds["t2m"].reset_coords(drop=True)
        da_list.append(
            da.expand_dims(period=[f"({clima_slice.start}-{clima_slice.stop})"])
        )
clima_da = xr.concat(clima_dataarrays, "period")
month_da = xr.concat(month_dataarrays, "period")
season_da = xr.concat(season_dataarrays, "period")

## Plot climatology maps

In [ ]:
for da_to_plot in (clima_da, season_da):
    da_to_plot = da_to_plot.isel(period=0)
    plot.projected_map(
        da_to_plot,
        projection=ccrs.Robinson(),
        cmap="YlOrRd",
        levels=9,
        robust=True,
        extend="both",
        col="season" if "season" in da_to_plot.dims else None,
        col_wrap=2,
    )
    plt.show()

## Plot anomaly maps

In [ ]:
for da_to_plot in (clima_da, season_da):
    da_to_plot = da_to_plot.drop_isel(period=0)
    label = " — ".join(da_to_plot["period"].values.tolist()[::-1][:2])
    with xr.set_options(keep_attrs=True):
        da_to_plot = da_to_plot.diff("period")
    da_to_plot.attrs["long_name"] += " anomaly"
    da_to_plot["period"] = [label]
    plot.projected_map(
        da_to_plot,
        projection=ccrs.Robinson(),
        cmap="RdBu_r",
        levels=9,
        center=0,
        robust=True,
        extend="both",
        col="season" if "season" in da_to_plot.dims else None,
        col_wrap=2,
    )
    plt.show()

## Plot seasonal cycle

In [ ]:
da_to_plot = diagnostics.spatial_weighted_mean(month_da)
da_to_plot.plot(hue="period")
plt.grid()

# Show table with values in Celsius
(da_to_plot - 273.15).round(1).to_pandas()